In [1]:
%cd ../

/home/users/dmoreno2016/VisionTransformers


In [17]:
import pandas as pd
import glob

In [3]:
colors= {
    0: '#00FF70',
    1: '#70FF00',
    2: '#FF0070',
    3: '#FF7000',
    4: '#0070FF',
    5: '#7000FF'
    }

In [18]:
dict_mapping_real_classes = {
6: 'Single u-lens',
15: 'TDE',
16: 'Eclip. Binary',
42: 'SNII',
52: 'SNIax',
53: 'Mira',
62: 'SNIbc',
64: 'Kilonova',
65: 'M-dwarf',
67: 'SNIa-91bg',
88: 'AGN',
90: 'SNIa',
92: 'RR lyrae',
95: 'SLSN-I',
#991: Others,
#992: Others,
#993: Others,
#994: Others, # Si lo considero, manualmente deberia cambiarlo en get_plasticc
}

In [20]:
path_data = 'data/lightcurves/plasticc/augmented_data/oversampling_10000_class'
snid_name = 'object_id'
target_name = 'target'
fold = 0

dict_mapping_classes= {
'RR lyrae': 0,
'AGN': 1,
'SNII': 2,
'SNIa': 3,
'M-dwarf': 4,
'Eclip. Binary': 5,
'SNIa-91bg': 6,
'SLSN-I': 7,
'SNIbc': 8,
'TDE': 9,
'SNIax': 10,
'Single u-lens': 11,
'Kilonova': 12,
'Mira': 13,
}


df_aug = []
df_obj_label_aux = []
for i, path in enumerate(glob.glob(f'{path_data}/fold_{fold}/*')):
    class_name = path.split('/')[-1].split('_')[0]
    print(class_name)
    df = pd.read_parquet(path)
    df_aug.append(df)
    unique_objs = df[snid_name].unique()
    df_obj_label_aux.append(pd.DataFrame({
        snid_name: unique_objs,
        target_name: [class_name] * len(unique_objs),
        'label_int': [dict_mapping_classes[class_name]] * len(unique_objs),
    }))

df_aug = pd.concat(df_aug)
df_obj_label_aux = pd.concat(df_obj_label_aux)

AGN
Eclip. Binary
SNIbc
SNIax
Kilonova
M-dwarf
SNIa-91bg
SNII
RR lyrae
SNIa
Single u-lens
Mira
SLSN-I
TDE


In [21]:
df_aug

,object_id,mjd,passband,flux,flux_err,detected
0,713_0,59825.2600,2,8.852702,1.013889,0
1,713_0,59825.2676,1,5.957424,1.160329,1
2,713_0,59825.2752,3,5.353736,1.932316,1
3,713_0,59825.2862,4,3.872437,3.018003,0
4,713_0,59825.2971,5,29.223808,7.163382,0
...,...,...,...,...,...,...
1244867,26400170_9603,60590.9951,4,3.621613,9.837766,0
1244868,26400170_9603,60599.0314,5,-3.133537,23.550060,0
1244869,26400170_9603,60611.0512,1,6.683933,1.589092,0
1244870,26400170_9603,60616.0201,4,1.500834,11.517282,0


In [22]:
df.object_id.nunique()

9604

In [23]:
df_obj_label_aux

,object_id,target,label_int
0,713_0,AGN,1
1,3489_1,AGN,1
2,4088_2,AGN,1
3,7315_3,AGN,1
4,7409_4,AGN,1
...,...,...,...
9599,35973028_9599,TDE,9
9600,120060791_9600,TDE,9
9601,61612630_9601,TDE,9
9602,9310078_9602,TDE,9


In [24]:
path_data = 'data/lightcurves/plasticc/raw'
snid_name = 'object_id'
target_name = 'target'

df_metadata_training = pd.read_csv(f'{path_data}/training_set_metadata.csv')
df_obj_label = df_metadata_training.copy()[[snid_name, target_name]]
df_metadata_training = df_metadata_training.drop([target_name], axis=1).set_index(snid_name)
df_obj_label[target_name] = df_obj_label[target_name].replace(dict_mapping_real_classes)
df_obj_label['label_int'] = df_obj_label[target_name].map(dict_mapping_classes)
df_obj_label

,object_id,target,label_int
0,615,RR lyrae,0
1,713,AGN,1
2,730,SNII,2
3,745,SNIa,3
4,1124,SNIa,3
...,...,...,...
7843,130739978,M-dwarf,4
7844,130755807,SNIa,3
7845,130762946,Eclip. Binary,5
7846,130772921,M-dwarf,4


In [25]:
path_partition = 'data/lightcurves/plasticc/partitions'
df_partitions = pd.read_parquet(f'{path_partition}/train_val/stratified_5fold_splits.parquet')
ids_partition = df_partitions[df_partitions['subset'] == f'train_{fold}'][snid_name].values
df_obj_label = df_obj_label[df_obj_label[snid_name].isin(ids_partition)]
df_obj_label

,object_id,target,label_int
1,713,AGN,1
4,1124,SNIa,3
5,1227,M-dwarf,4
6,1598,SNIa,3
7,1632,SNII,2
...,...,...,...
7841,130716752,Eclip. Binary,5
7843,130739978,M-dwarf,4
7844,130755807,SNIa,3
7845,130762946,Eclip. Binary,5


In [26]:
pd.concat([df_obj_label_aux, df_obj_label]).groupby('target').count()['object_id']

target
AGN              10000
Eclip. Binary    10000
Kilonova         10000
M-dwarf          10000
Mira             10000
RR lyrae         10000
SLSN-I           10000
SNII             10000
SNIa             10000
SNIa-91bg        10000
SNIax            10000
SNIbc            10000
Single u-lens    10000
TDE              10000
Name: object_id, dtype: int64

In [27]:
df_lc = pd.read_csv(f'{path_data}/training_set.csv')
df_lc

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1
...,...,...,...,...,...,...
1421700,130779836,60555.9838,4,-39.881969,46.477093,0
1421701,130779836,60560.0459,1,14.894439,18.947685,0
1421702,130779836,60571.0225,5,30.593130,50.695290,0
1421703,130779836,60585.9974,4,-23.471439,44.819859,0


In [110]:
df_lc.flux.max()

2432808.75

In [108]:
df_lc[df_lc.object_id == 56949500]

,object_id,mjd,passband,flux,flux_err,detected
1005193,56949500,59583.2996,0,-375.556885,28.511673,1
1005194,56949500,59583.3547,4,-1587.850098,30.664392,1
1005195,56949500,59589.0603,5,-1443.190552,42.813660,1
1005196,56949500,59590.3313,3,3374.122803,47.980717,1
1005197,56949500,59593.0513,4,-1379.019531,22.650082,1
...,...,...,...,...,...,...
1005329,56949500,60658.1807,4,-1789.058350,25.261791,1
1005330,56949500,60665.1383,3,3447.164795,44.441593,1
1005331,56949500,60666.1067,1,-2714.734619,62.800926,1
1005332,56949500,60667.1600,1,-2641.711670,60.541199,1


In [27]:
df_obj_label.groupby('target').count()['object_id']

target
6      151
15     495
16     924
42    1193
52     183
53      30
62     484
64     102
65     981
67     208
88     370
90    2313
92     239
95     175
Name: object_id, dtype: int64

In [28]:
path_partition = 'data/lightcurves/plasticc/partitions'
df_partitions = pd.read_parquet(f'{path_partition}/train_val/stratified_5fold_splits.parquet')
df_partitions

,object_id,target,subset
0,615,RR lyrae,val_0
1,730,SNII,val_0
2,745,SNIa,val_0
3,1920,SNIa,val_0
4,2330,SNIa,val_0
...,...,...,...
39235,130716752,Eclip. Binary,train_4
39236,130727624,M-dwarf,train_4
39237,130755807,SNIa,train_4
39238,130772921,M-dwarf,train_4


In [29]:
df_train_0 = df_partitions[df_partitions.subset == 'train_0']
df_train_0

,object_id,target,subset
1570,713,AGN,train_0
1571,1124,SNIa,train_0
1572,1227,M-dwarf,train_0
1573,1598,SNIa,train_0
1574,1632,SNII,train_0
...,...,...,...
7843,130716752,Eclip. Binary,train_0
7844,130739978,M-dwarf,train_0
7845,130755807,SNIa,train_0
7846,130762946,Eclip. Binary,train_0


In [76]:
object_ids = list(df_train_0[df_train_0.target == 'SNII'].object_id.values)
object_ids

[1632,
 2103,
 2300,
 3285,
 4132,
 4220,
 5527,
 7033,
 10798,
 11359,
 25783,
 29252,
 29668,
 31824,
 32309,
 33191,
 34166,
 35197,
 35743,
 36085,
 38205,
 40290,
 41738,
 42224,
 43028,
 45319,
 45549,
 48260,
 48473,
 52425,
 54416,
 55033,
 55060,
 58174,
 59068,
 59463,
 63860,
 64896,
 66126,
 66852,
 67245,
 67898,
 68835,
 69490,
 70135,
 70430,
 71084,
 73509,
 75598,
 77623,
 77825,
 79155,
 79921,
 80780,
 81000,
 81464,
 86456,
 93509,
 95690,
 96284,
 98570,
 99050,
 99862,
 100097,
 100331,
 102036,
 102864,
 103026,
 103967,
 104526,
 107615,
 108487,
 109860,
 110241,
 111281,
 111799,
 113206,
 114341,
 115053,
 119383,
 120356,
 121224,
 121266,
 121705,
 125743,
 127056,
 128488,
 129490,
 130220,
 130404,
 131075,
 135097,
 136407,
 137645,
 138010,
 138068,
 138947,
 141302,
 142866,
 142867,
 143066,
 145926,
 148535,
 149478,
 150561,
 150880,
 151427,
 152300,
 154053,
 155778,
 157120,
 157299,
 158515,
 162139,
 163208,
 165821,
 166186,
 167310,
 170135,


In [77]:
len(object_ids)

954

In [78]:
import numpy as np

num_total = len(object_ids) + 200

if num_total <= len(object_ids):
    augmented_ids = object_ids[:num_total]
else:
    # Calculate how many times to repeat the original IDs
    num_repeats = num_total // len(object_ids)
    remainder = num_total % len(object_ids)
    
    # Repeat the list and add the remaining IDs
    augmented_ids = np.tile(object_ids, num_repeats).tolist()
    augmented_ids += object_ids[:remainder]
augmented_ids

[1632,
 2103,
 2300,
 3285,
 4132,
 4220,
 5527,
 7033,
 10798,
 11359,
 25783,
 29252,
 29668,
 31824,
 32309,
 33191,
 34166,
 35197,
 35743,
 36085,
 38205,
 40290,
 41738,
 42224,
 43028,
 45319,
 45549,
 48260,
 48473,
 52425,
 54416,
 55033,
 55060,
 58174,
 59068,
 59463,
 63860,
 64896,
 66126,
 66852,
 67245,
 67898,
 68835,
 69490,
 70135,
 70430,
 71084,
 73509,
 75598,
 77623,
 77825,
 79155,
 79921,
 80780,
 81000,
 81464,
 86456,
 93509,
 95690,
 96284,
 98570,
 99050,
 99862,
 100097,
 100331,
 102036,
 102864,
 103026,
 103967,
 104526,
 107615,
 108487,
 109860,
 110241,
 111281,
 111799,
 113206,
 114341,
 115053,
 119383,
 120356,
 121224,
 121266,
 121705,
 125743,
 127056,
 128488,
 129490,
 130220,
 130404,
 131075,
 135097,
 136407,
 137645,
 138010,
 138068,
 138947,
 141302,
 142866,
 142867,
 143066,
 145926,
 148535,
 149478,
 150561,
 150880,
 151427,
 152300,
 154053,
 155778,
 157120,
 157299,
 158515,
 162139,
 163208,
 165821,
 166186,
 167310,
 170135,


In [79]:
len(augmented_ids)

1154

In [32]:
df_train_0.groupby('target').count()['object_id'].max()

1851

In [23]:
df_train_0.groupby('target').count()['object_id']

target
AGN               296
Eclip. Binary     740
Kilonova           82
M-dwarf           784
Mira               24
RR lyrae          191
SLSN-I            140
SNII              954
SNIa             1851
SNIa-91bg         166
SNIax             147
SNIbc             387
Single u-lens     120
TDE               396
Name: object_id, dtype: int64

In [71]:
num_aug_by_class = df_train_0.groupby('target').count()['object_id'].max() - \
    df_train_0.groupby('target').count()['object_id']
num_aug_by_class

target
AGN              1555
Eclip. Binary    1111
Kilonova         1769
M-dwarf          1067
Mira             1827
RR lyrae         1660
SLSN-I           1711
SNII              897
SNIa                0
SNIa-91bg        1685
SNIax            1704
SNIbc            1464
Single u-lens    1731
TDE              1455
Name: object_id, dtype: int64

In [72]:
num_aug_by_class.sum()

19636

In [62]:
df_lc = pd.read_csv(f'{path_data}/training_set.csv')
df_lc

,object_id,mjd,passband,flux,flux_err,detected
0,615,59750.4229,2,-544.810303,3.622952,1
1,615,59750.4306,1,-816.434326,5.553370,1
2,615,59750.4383,3,-471.385529,3.801213,1
3,615,59750.4450,4,-388.984985,11.395031,1
4,615,59752.4070,2,-681.858887,4.041204,1
...,...,...,...,...,...,...
1421700,130779836,60555.9838,4,-39.881969,46.477093,0
1421701,130779836,60560.0459,1,14.894439,18.947685,0
1421702,130779836,60571.0225,5,30.593130,50.695290,0
1421703,130779836,60585.9974,4,-23.471439,44.819859,0


In [ ]:
def gaussian_augmentation(obj_id, n_samples=5):
    df = df[df['object_id'] == obj_id]

    augmented_data = []
    for index, row in df.iterrows():
        samples = np.random.normal(loc=row['flux'], scale=row['flux_err'], size=n_samples)
        for i, sample in enumerate(samples, 1):  # Start enumeration at 1 for augmented sample ID
            augmented_data.append({
                'object_id': f"{int(row['object_id'])}_{i}",
                'mjd': row['mjd'],
                'flux': sample,
                'flux_err': row['flux_err'], # Unique ID including row index and sample number
            })
    return pd.DataFrame(augmented_data)


# Define the function to generate all augmented data for a single astronomical object
def augment_object(args):
    df, n_samples = args
    augmented_data = []
    for index, row in df.iterrows():
        samples = np.random.normal(loc=row['flux'], scale=row['flux_err'], size=n_samples)
        for i, sample in enumerate(samples, 1):  # Start enumeration at 1 for augmented sample ID
            augmented_data.append({
                'object_id': f"{int(row['object_id'])}_{i}",
                'mjd': row['mjd'],
                'flux': sample,
                'flux_err': row['flux_err']
            })
    return pd.DataFrame(augmented_data)

for label, num_aug in num_aug_by_class.items():
    print(f'We are augmentating the {label} class')
    object_ids = df_train_0[df_train_0.target == label].object_id.values
    args_list = [(df_lc[df_lc['object_id'] == obj_id], num_aug) for obj_id in object_ids]

    print(f'Multiprocessing started...')
    with Pool() as pool:
        results = pool.map(augment_object, args_list)

    df_augmented = pd.concat(results, ignore_index=True)

In [64]:
def gaussian_augmentation(df, n_samples=5):
    augmented_data = []
    for index, row in df.iterrows():
        samples = np.random.normal(loc=row['flux'], scale=row['flux_err'], size=n_samples)
        for i, sample in enumerate(samples, 1):  # Start enumeration at 1 for augmented sample ID
            augmented_data.append({
                'object_id': f"{int(row['object_id'])}_{i}",
                'mjd': row['mjd'],
                'flux': sample,
                'flux_err': row['flux_err'], # Unique ID including row index and sample number
            })
    return pd.DataFrame(augmented_data)

df_augmented = []
for label, num_aug in num_aug_by_class.items():
    print(f'We are augmentating the {label} class')
    ids_class = df_train_0[df_train_0.target == label].object_id.values
    df_lc_aux = df_lc[df_lc.object_id.isin(ids_class)]
    df_lc_aux = gaussian_augmentation(df_lc_aux, n_samples=5)
    df_augmented.append(df_lc_aux)

We are augmentating the AGN class
We are augmentating the Eclip. Binary class
We are augmentating the Kilonova class
We are augmentating the M-dwarf class
We are augmentating the Mira class
We are augmentating the RR lyrae class
We are augmentating the SLSN-I class
We are augmentating the SNII class
We are augmentating the SNIa class
We are augmentating the SNIa-91bg class
We are augmentating the SNIax class
We are augmentating the SNIbc class
We are augmentating the Single u-lens class
We are augmentating the TDE class


In [65]:
df_augmented = pd.concat(df_augmented, ignore_index=True)
df_augmented

,object_id,mjd,flux,flux_err
0,713_1,59825.2600,8.662634,1.013889
1,713_2,59825.2600,9.427334,1.013889
2,713_3,59825.2600,10.725931,1.013889
3,713_4,59825.2600,10.826805,1.013889
4,713_5,59825.2600,9.263082,1.013889
...,...,...,...,...
5685315,130695262_1,60670.0746,-0.214617,2.026918
5685316,130695262_2,60670.0746,3.344613,2.026918
5685317,130695262_3,60670.0746,2.263977,2.026918
5685318,130695262_4,60670.0746,-0.442906,2.026918


In [66]:
df_augmented[df_augmented.object_id == '713_1']

,object_id,mjd,flux,flux_err
0,713_1,59825.2600,8.662634,1.013889
5,713_1,59825.2676,7.387935,1.160329
10,713_1,59825.2752,7.302603,1.932316
15,713_1,59825.2862,-1.321149,3.018003
20,713_1,59825.2971,14.658229,7.163382
...,...,...,...,...
1725,713_1,60668.1165,-14.049676,5.354884
1730,713_1,60671.0655,-10.058612,1.726118
1735,713_1,60672.0693,-13.620286,1.470152
1740,713_1,60673.0745,-11.901718,2.243120


In [ ]:
# Applying the Gaussian augmentation
#df_augmented = gaussian_augmentation(df_lc, n_samples=5)
#df_augmented

## Other Augmentations

In [12]:
import numpy as np

# Adding Noise: Introduce random Gaussian noise to the flux values. This can help the model become more robust to small fluctuations in the data.
def add_noise(df, noise_level=0.01):
    noise = np.random.normal(0, noise_level, size=df['flux'].shape)
    df['flux_aug'] = df['flux'] + noise * df['flux_err']
    return df

# Scaling: Multiply the flux values by a constant factor to simulate different brightness levels.
def scale_flux(df, scale_factor=1.1):
    df['flux_aug'] = df['flux'] * scale_factor
    return df

# Random Cropping: This involves selecting a random subsection of the time series. This is useful for models to learn from different parts of the light curve.
def random_crop(df, crop_size=50):
    start = np.random.randint(0, len(df) - crop_size)
    return df.iloc[start:start + crop_size]

# Time Warping: Apply a slight stretching or compression to the time axis, which can simulate the effect of different observational cadences.
def time_warp(df, warp_factor=0.9):
    time_steps = np.diff(df['mjd']) * warp_factor
    df['mjd_aug'] = np.concatenate([[df['mjd'].iloc[0]], df['mjd'].iloc[0] + np.cumsum(time_steps)])
    return df

# Jittering: Randomly adjust the flux values within the uncertainty given by flux_err.
def jitter_flux(df):
    jitter = np.random.uniform(-1, 1, size=df.shape[0]) * df['flux_err']
    df['flux_aug'] = df['flux'] + jitter
    return df

# Inverting: Flip the flux values around the median flux, which can simulate viewing the light curve from different angles or opposite phases.
def invert_flux(df):
    median_flux = df['flux'].median()
    df['flux_aug'] = 2 * median_flux - df['flux']
    return df

In [13]:
# Apply augmentation
df_augmented = add_noise(df_lc)
df_augmented

,object_id,mjd,passband,flux,flux_err,detected,flux_aug
0,615,59750.4229,2,-544.810303,3.622952,1,-544.778625
1,615,59750.4306,1,-816.434326,5.553370,1,-816.476936
2,615,59750.4383,3,-471.385529,3.801213,1,-471.391557
3,615,59750.4450,4,-388.984985,11.395031,1,-389.006063
4,615,59752.4070,2,-681.858887,4.041204,1,-681.862797
...,...,...,...,...,...,...,...
1421700,130779836,60555.9838,4,-39.881969,46.477093,0,-39.725409
1421701,130779836,60560.0459,1,14.894439,18.947685,0,14.687932
1421702,130779836,60571.0225,5,30.593130,50.695290,0,30.889442
1421703,130779836,60585.9974,4,-23.471439,44.819859,0,-24.107361
